# **Creating a Neural Search Method**
Now that I've got some helpful methods within the **`01. Writing Postgres Queries`** notebook, I can spend some time writing a "neural search" method. This will be used for the API. 

# Setup
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [1]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

d:\data\programming\neural-needledrop\api


Now I'll import some necessary modules:

In [2]:
# General import statements
import pandas as pd
import datetime
from IPython.display import Markdown, display

# Importing custom modules
from utils.settings import (
    POSTGRES_USER,
    POSTGRES_PASSWORD,
    POSTGRES_HOST,
    POSTGRES_PORT,
    POSTGRES_DB,
)
import utils.postgres_queries as pg_queries
import utils.postgres as postgres
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base

I'll also set up my connection to the Postgres server: 

In [3]:
# Create the connection string to the database
postgres_connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the connection engine
engine = create_engine(postgres_connection_string)
metadata = MetaData()
session = sessionmaker(bind=engine)()
Base = declarative_base()

# **Prototyping Neural Search**
Below, I'm going to prototype a search method. 

I'll start by parameterizing the search:

In [4]:
# Parameterize the search
query = "Love Me Forever, which was a very nice, airy, euphoric ballad, is just a minute and change"
release_date_filter = [datetime.datetime(2010, 1, 1), datetime.datetime(2024, 6, 1)]
video_type_filter = ["album_review", "mixtape_review"]
review_score_filter = [8, 10]

# Extra parameters
n_chunks_to_consider_initially = 250
n_most_similar_chunks_per_video = 10
n_videos_to_return = 10
n_segment_chunks_to_showcase = 3

Now, I'll try and identify the most similar segments:

In [5]:
# Run the query
similar_chunks_df = pg_queries.most_similar_embeddings_to_text_filtered(
    text=query,
    engine=engine,
    n=n_chunks_to_consider_initially,
    release_date_filter=release_date_filter,
    video_type_filter=video_type_filter,
    review_score_filter=review_score_filter,
    include_text=True,
)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column text_segments_to_fetch.video_url does not exist
LINE 10:             text_segments_to_fetch.video_url = transcription...
                     ^

[SQL: 
            SELECT
            transcriptions.*,
            text_segments_to_fetch.embedding_id
            FROM
            transcriptions
            JOIN
            text_segments_to_fetch
            ON
            text_segments_to_fetch.video_url = transcriptions.url
            ]
(Background on this error at: https://sqlalche.me/e/20/f405)

With this in hand, I want to aggregate across a bunch of videos, and try to determine which video has the highest scores.

In [ ]:
# Groupby `url`, and take the top `n_most_similar_chunks_per_video` chunks per video
aggregated_similar_chunks_df = similar_chunks_df.groupby("url").head(
    n_most_similar_chunks_per_video
)

# Aggregate the similarity statistics
aggregated_similar_chunks_df = (
    aggregated_similar_chunks_df.groupby("url")
    .agg(
        median_similarity=("cos_sim", "median"),
        n_similar_chunks=("cos_sim", "count"),
    )
    .reset_index()
)

# Add a weighted median similarity column
aggregated_similar_chunks_df["weighted_median_similarity"] = (
    aggregated_similar_chunks_df["median_similarity"]
    * aggregated_similar_chunks_df["n_similar_chunks"]
)

# Sort by the weighted median similarity
aggregated_similar_chunks_df = aggregated_similar_chunks_df.sort_values(
    "weighted_median_similarity", ascending=False
).head(n_videos_to_return)

Now, we're going to get some metadata about each video back. This will involve uploading a temporary table to Postgres, and then joining it to the `video_metadata` table:

In [ ]:
# Create a temporary table called `temp_similar_chunks` that is the aggregated_similar_chunks_df DataFrame
with engine.connect() as conn:
    aggregated_similar_chunks_df.to_sql(
        "temp_similar_chunks", conn, if_exists="replace", index=False
    )

# Now, select the entire `video_metadata` table for each of the videos in the `temp_similar_chunks` table
similar_chunks_video_metadata_df = postgres.query_postgres(
    """
    SELECT 
        video_metadata.*, 
        temp_similar_chunks.median_similarity, 
        temp_similar_chunks.n_similar_chunks, 
        temp_similar_chunks.weighted_median_similarity
    FROM video_metadata
    JOIN temp_similar_chunks
    ON video_metadata.url = temp_similar_chunks.url
    ORDER BY temp_similar_chunks.weighted_median_similarity DESC
    """,
    engine=engine,
)

Finally, we're going to prepare our results. This will just be the `similar_chunks_video_metadata_df`, except also containing the `n_segment_chunks_to_showcase` most similar segment chunks per video. 

In [ ]:
# Create a DataFrame containing the segment chunks I want to showcase
segment_chunks_to_showcase_df = (
    (
        similar_chunks_df[
            similar_chunks_df["url"].isin(
                similar_chunks_video_metadata_df["url"].unique()
            )
        ]
        .sort_values("cos_sim", ascending=False)
        .groupby("url")
        .head(n_segment_chunks_to_showcase)
        .sort_values(["url", "cos_sim"], ascending=False)
    )
    .groupby("url")
    .agg(
        top_segment_chunks=("text", lambda x: list(x)),
    )
    .reset_index()
)

# Merge this DataFrame with the video metadata
segment_chunks_to_showcase_df = segment_chunks_to_showcase_df.merge(
    similar_chunks_video_metadata_df, on="url"
).sort_values("weighted_median_similarity", ascending=False)

Now, I'm going to print the results using some nice formatting:

In [ ]:
for index, row in segment_chunks_to_showcase_df.head(3).iterrows():
    display(Markdown(f"**{row['title']}**"))
    display(Markdown("\n".join([f"* {chunk}" for chunk in row['top_segment_chunks']])))


# **Functionalized Version**
I took all of the code above and wrote a single method from it. Below, I'll show it off:

In [ ]:
from utils.search import neural_search

# Run the search
neural_search(
    query="fuzzy muted math rock guitar licks",
    release_date_filter=None,
    video_type_filter=None,
    review_score_filter=None,
    n_most_similar_chunks_per_video=8,
    n_videos_to_return=3,
    n_segment_chunks_to_showcase=3,
)